In [1]:
from google.colab import drive

In [2]:
pwd

'/content'

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
cd /content/drive/MyDrive/ColapML/TransferLearning

/content/drive/MyDrive/ColapML/TransferLearning


In [5]:
pwd

'/content/drive/MyDrive/ColapML/TransferLearning'

In [6]:
!pip3 install wget


  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9681 sha256=4085e6c58634568bfc526798b4b49c5a407162efecbcda5ae20df3ee0c79e236
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [7]:
import wget

In [8]:
# cái cấu trúc của mạng nó sẻ nằm luôn ở trong cái thư viên này 
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [9]:
local_weights_file = './inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

In [10]:
import os
from tensorflow.keras import layers
from tensorflow.keras import models


In [11]:
# include_top = false: Nghĩa là chúng ta có cái lớp fully-connected hay soft max chúng ta sẻ bỏ đi
# weight chúng ta tải về củng no-top luôn
pre_trained_model = InceptionV3(input_shape=(150,150,3), include_top=False, weights=None)

In [12]:
pre_trained_model.load_weights(local_weights_file)

In [13]:

pre_trained_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 74, 74, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 74, 74, 32)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 74, 74, 32)   0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [14]:
# chúng ta sẻ đóng băng ở đâu lấy chổ nào để làm fixed function 
# chúng ta tạm thời đóng băng tất cả lại, khổng  thể train những cái layer này 
for layer in pre_trained_model.layers:
  layer.trainable = False

In [15]:
pre_trained_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 74, 74, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 74, 74, 32)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 74, 74, 32)   0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [16]:

last_layer = pre_trained_model.get_layer('mixed7')

In [17]:
last_output = last_layer.output

In [18]:
# lấy ra cái last_output của cái last_layer nó chính là cái ma trận 4 chiều 
last_output

<tf.Tensor 'mixed7/concat:0' shape=(None, 7, 7, 768) dtype=float32>

In [19]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import Model

In [20]:
x = layers.Flatten()(last_output)


In [21]:
x = layers.Dense(1024, activation='relu')(x)


In [22]:
# 0.2 là 20% 
# Dropout là để chống overfiting
# trong trường hợp này chúng ta tắt đi 20% việc tắt này sẻ là ngẫu nhiên
# x chính là toàn bộ cái lớp đóng băng và cái lớp chúng ta đã thêm
x = layers.Dropout(0.2)(x)


In [23]:
# Do củng chỉ có 2 class thôi nên nếu lớn hơn 0.5 thì là con gì nhỏ hơn 0.5 là con gì
x = layers.Dense(1, activation='sigmoid')(x)


In [24]:
# chúng ta sẻ có last_layer và thêm những cái layer mới ở trên
#chúng ta sẻ tạo ra 1 cái model mới 
# và cài input
model = Model(pre_trained_model.input, x)
model.compile(optimizer=RMSprop(lr=0.0001), loss='binary_crossentropy', metrics=['acc'])

In [25]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 74, 74, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 74, 74, 32)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 74, 74, 32)   0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [26]:
# Trainable params chính là bằng cái params của cái lớp Dense mà ra 

In [27]:
train_folder = './cats_and_dogs_filtered/train'
valid_folder = './cats_and_dogs_filtered/validation'

In [28]:
pwd

'/content/drive/My Drive/ColapML/TransferLearning'

In [29]:
wget.download('https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip')

'cats_and_dogs_filtered (1).zip'

In [30]:
!unzip -q cats_and_dogs_filtered.zip -d cats_and_dogs_filtered

In [31]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [32]:

train_datagen = ImageDataGenerator(rotation_range=40,width_shift_range=0.2,height_shift_range=0.2,horizontal_flip=True,rescale=1./255, shear_range= 0.2, zoom_range=0.2)

In [33]:
# do không cần train nền  không cần tăng cường ảnh
valid_datagen = ImageDataGenerator(rescale=1.0/255)

In [34]:
train_generator = train_datagen.flow_from_directory(directory=train_folder,batch_size=20, class_mode='binary', target_size=(150,150))
valid_generator = valid_datagen.flow_from_directory(directory=valid_folder, batch_size=20, class_mode='binary', target_size = (150,150))

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [35]:
#history = model.fit_generator(train_generator, validation_data=valid_generator, steps_per_epoch=100, epochs=20,validation_steps=50,verbose=2)

In [36]:
model.save('model.h5')

In [38]:
import tensorflow as tf

In [39]:
model = tf.keras.models.load_model('model.h5')

In [40]:
train_generator.class_indices

{'cats': 0, 'dogs': 1}

In [46]:
import numpy as np
from google.colab import files
from keras.preprocessing import image
uploaded = files.upload()
for fn in uploaded.keys():
  path = './' + fn
  # load path vào và lấy ra cái array và sửa cái hình thành 150x150
  img = image.load_img(path, target_size=(150,150))
  # chuẩn hóa lại nó 
  x = image.img_to_array(img)/255
  # tăng thêm cho nó 1 chiều nữa để thành 4 chiều
  # Tại sao 4 chiều ? tài vì cái input_shape của mình yêu cầu là 4 chiều
  x = np.expand_dims(x,axis = 0 )
  # vstack: để gộp mảng
  images = np.vstack([x])
  classes= model.predict(images,batch_size =10)
  if(classes[0] > 0.5):
    print(fn + " is a dogs")
  else:
    print(fn + "is a cats")

Saving cat.2495.jpg to cat.2495 (1).jpg
cat.2495.jpg is a dogs
